In [6]:
##### This code is from Updated Testing Reddit - No Con- bias (Fictitious Play)-01092022
##### This code replace the big real datanetwork with small sythetic network 
import scipy
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations
import time
import random
from scipy.stats import beta
import pandas as pd
import copy
%matplotlib inline
#%run Helpers.ipynb
#%run pure_strategy_selection.ipynb  #include simple selection algorithm
import scipy.io
import collections
import sys

    # centers the opinion vector around 0\n",
def mean_center(op, n):
    ones = np.ones((n, 1))
    x = op - (np.dot(np.transpose(op),ones)/n) * ones
    return x
    
# compute number of edges, m\n
def num_edges(L, n):
    m = 0
    for i in range(n):
        for j in range(n):
            if i > j and L[i,j] < 0:
                m += 1            
    return m

# maximizing polarization only: \\bar{z}^T \\bar{z}   
def obj_polarization(A, op, n):
    op_mean = mean_center(op, n)
    z_mean = np.dot(A, op_mean) 
    return np.dot(np.transpose(z_mean), z_mean)[0,0] 

def obj_polarization_1(A, op, n):
    z = np.dot(A, op) 
    z_mean = mean_center(z, n)
    return np.dot(np.transpose(z_mean), z_mean)[0,0] 

# Calculate innate polarization
def obj_innate_polarization(s, n):  
#     np.set_printoptions(precision=5)
    op_mean = mean_center(s, n)
    return np.dot(np.transpose(op_mean), op_mean)[0,0]

In [7]:
# Parameters for the network

np.set_printoptions(precision=3)

In [10]:
def make_sync_network(n):
    # def make_innat_opinions(n): # Make opinion for agents only - no info source is involved     
    # Make list of ind innate opinion to define info source opinion
    innat_s = np.random.uniform(low=0, high=1, size=int(n))   #individual's innate opinion 

    s = np.zeros((n, 1))
    
    idx1 = 0
    for i in range(len(s)):
        s[i] = innat_s[idx1]  #set innate opinion for ind.
        idx1 += 1
  
        # return s

    # def make_random_network(n):
    # Create empty graph
    nxG = nx.Graph()
    # Add nodes to graph
    nxG.add_nodes_from(range(n))
    # Add edges based on node index
    for i in range(n):
        for j in range(i):
            p = (n-i)/(n-j) # Probability of edge existing
            if np.random.random() < p:
                nxG.add_edge(i, j)
    
        # Draw graph
        #nx.draw(nxG, with_labels=True)
        #plt.show()
    G = nx.adjacency_matrix(nxG).todense()
        
        # return G

    # s = make_innat_opinions(n)
    # G = make_random_network(n) 
    L = scipy.sparse.csgraph.laplacian(G, normed=False)  # Return the Laplacian matrix
    A = np.linalg.inv(np.identity(n) + L)  # A = (I + L)^(-1)\n  Stanford paper theory

    return s,G,A


## Node properties

In [71]:
# Creat Metrics for Regression

################### create shortest path from all nodes to max selected node v1 ############
def shortest_pth(G, v1):
    nxG = nx.from_numpy_array(G)  
    paths = nx.single_source_shortest_path(nxG, v1) 
    # the length of shortest path from v2 to v1
    lenths =[]
    for v in paths.items():
        lenth = len(v[1])
        lenths.append(lenth)  
    PathLen = dict(zip(paths.keys(),lenths))
    #print("path_lenth",PathLen)
    return PathLen
    
def local_clustering_coefficient(G):
    clus_coef = []
    for node in range(len(G)):
        neighbors = np.nonzero(G[node])[0]       
        num_neighbors = len(neighbors)
        if num_neighbors < 2:
            clustering_coefficient = 0.0  # No triangles possible
        else:
            num_triangles = 0
            for i in range(num_neighbors - 1):
                for j in range(i + 1, num_neighbors):
                    #print(neighbors[i], neighbors[i])
                    if G[neighbors[i], neighbors[j]] == 1:
                        num_triangles = num_triangles + 1
                        #print(num_triangles)
#             print("num_triangles:",num_triangles)
            clustering_coefficient = (2.0 * num_triangles) / (num_neighbors * (num_neighbors - 1))
        clus_coef.append((node, clustering_coefficient))
    cc = dict(clus_coef)
    return cc

def metrics(G, v1):

    nxG = nx.from_numpy_array(G) 
    # G = nx.karate_club_graph()
    deg_centrality = nx.degree_centrality(nxG)
    close_centrality = nx.closeness_centrality(nxG)
    pr = nx.eigenvector_centrality(nxG)      # page rank 
    PathLen = shortest_pth(G, v1)  #Shortest Path_
    cc = local_clustering_coefficient(G)  #cluster coefficient

   # Opinion Extremity
    def gap(op, n):
        ones = np.ones((n, 1))
        x = op - (np.dot(np.transpose(op),ones)/n) * ones
        return abs(x)
    gap = gap(s,n)
    my_gap = {index: value for index, value in enumerate(gap)}
   
    # sorting all varibles based on the node index
    MinPaths_sort = dict(sorted(PathLen.items(), key=lambda x:x[1]))
    deg_cent = dict(sorted(deg_centrality.items(), key=lambda x:x[1]))
    clo_cent = dict(sorted(close_centrality.items(), key=lambda x:x[1]))
    eigen_cent = dict(sorted(pr.items(), key=lambda x:x[1]))
    op_extre = dict(sorted(my_gap.items(), key=lambda x:x[1]))
    cc = dict(sorted(cc.items(), key=lambda x:x[1]))
    
    return (deg_cent, clo_cent, eigen_cent, MinPaths_sort, op_extre, cc)

def top_k(input_dict):
    first_k_keys = list(input_dict.keys())[:k]

    return first_k_keys

